In [1]:
import os
import numpy as np
import pandas as pd
import time
Match=pd.read_csv('DataSets/Match.csv')
Match.index=Match.id

Players=pd.read_csv('DataSets/Player.csv')


PlayerAt=pd.read_csv('DataSets/Player_Attributes.csv')


Team=pd.read_csv('DataSets/Team.csv')


Total=pd.read_csv('DataSets/MatchData2.csv',sep=';')
Total.loc[:,'DateTime']=pd.to_datetime(Total['Date'],format='%d/%m/%Y')

Data=Match[['id']].copy()

Data.columns=['Match_ID']
Data.loc[:,'DateTime']=Match['date']

Data.loc[:,'HomeScore']=Match['home_team_goal']
Data.loc[:,'AwayScore']=Match['away_team_goal']

Data.loc[:,'Result']=0
Data.loc[ Data['HomeScore'] > Data['AwayScore'] ,'Result']=1
Data.loc[ Data['HomeScore'] < Data['AwayScore'] ,'Result']=-1
Data.loc[:,'Home_ID']=Match['home_team_api_id']
Data.loc[:,'Away_ID']=Match['away_team_api_id']





for i in range(1,12):
    Data.loc[:,'HomePlayer_'+str(i)]=Match['home_player_'+str(i)]

for i in range(1,12):
    Data.loc[:,'AwayPlayer_'+str(i)]=Match['away_player_'+str(i)]



ColumnNames=['HomePlayer_1', 'HomePlayer_2', 'HomePlayer_3', 'HomePlayer_4',
       'HomePlayer_5', 'HomePlayer_6', 'HomePlayer_7', 'HomePlayer_8',
       'HomePlayer_9', 'HomePlayer_10', 'HomePlayer_11', 'AwayPlayer_1',
       'AwayPlayer_2', 'AwayPlayer_3', 'AwayPlayer_4', 'AwayPlayer_5',
       'AwayPlayer_6', 'AwayPlayer_7', 'AwayPlayer_8', 'AwayPlayer_9',
       'AwayPlayer_10', 'AwayPlayer_11']





for col in ColumnNames:
    Data.loc[:,col+'_Overall'] = 0

    startTime=time.time()
    for index in Data.index:

        PlayerID=Data[col].loc[index]
        MatchDate=Data['DateTime'].loc[index]
        PlayerTable = PlayerAt[PlayerAt['player_api_id'] == PlayerID]
        PlayerTable2 = Players[Players['player_api_id'] == PlayerID]
        PlayerTable=PlayerTable.sort_values(by=['date'])
        try:
            PlayerTable=PlayerTable[PlayerTable['date']<MatchDate].iloc[-1]

            Data.at[index, col + '_Overall'] =PlayerTable['overall_rating']
        except:
            Data.at[index, col + '_Overall'] =None
    EndTime=time.time()
    print(col+' is completed in '+str(EndTime-startTime)+' seconds')



# Home Team Last Summary
for index in Data.index:
    HomeID=Data.loc[index,'Home_ID']
    MatchDate=Data.loc[index,'DateTime']
    TeamMatch=Match[(Match['home_team_api_id']==HomeID) |  (Match['away_team_api_id']==HomeID)]
    TeamMatch=TeamMatch.sort_values(by=['date'])
    TeamMatch=TeamMatch[TeamMatch['date']<MatchDate].iloc[-5:]
    GiveGoal=0
    TakeGoal=0
    WinNumber=0

    for indexTeam in TeamMatch.index:
        if TeamMatch.loc[indexTeam,'home_team_api_id']==HomeID:
            MatchGiveGoal =  TeamMatch.loc[indexTeam,'home_team_goal']
            MatchTakeGoal = TeamMatch.loc[indexTeam,'away_team_goal']
            GiveGoal=GiveGoal+MatchGiveGoal
            TakeGoal= TakeGoal+MatchTakeGoal

            if MatchGiveGoal>MatchTakeGoal:
                WinNumber = WinNumber+1

        else:
            MatchGiveGoal =  TeamMatch.loc[indexTeam, 'away_team_goal']
            MatchTakeGoal = TeamMatch.loc[indexTeam,'home_team_goal']
            GiveGoal = GiveGoal + MatchGiveGoal
            TakeGoal = TakeGoal + MatchTakeGoal


            if MatchGiveGoal > MatchTakeGoal:
                WinNumber = WinNumber + 1

    Data.at[index,'Last_5_match_goal_diff_home '] = GiveGoal-TakeGoal
    Data.at[index,'Last_5_match_win_home  '] = WinNumber


#Away Team
for index in Data.index:
    AwayID=Data.loc[index,'Away_ID']
    MatchDate=Data.loc[index,'DateTime']
    TeamMatch=Match[(Match['away_team_api_id']==AwayID) |  (Match['away_team_api_id']==AwayID)]
    TeamMatch=TeamMatch.sort_values(by=['date'])
    TeamMatch=TeamMatch[TeamMatch['date']<MatchDate].iloc[-5:]
    GiveGoal=0
    TakeGoal=0
    WinNumber=0

    for indexTeam in TeamMatch.index:
        if TeamMatch.loc[indexTeam,'home_team_api_id']==HomeID:
            MatchGiveGoal =  TeamMatch.loc[indexTeam,'home_team_goal']
            MatchTakeGoal = TeamMatch.loc[indexTeam,'away_team_goal']
            GiveGoal=GiveGoal+MatchGiveGoal
            TakeGoal= TakeGoal+MatchTakeGoal

            if MatchGiveGoal>MatchTakeGoal:
                WinNumber = WinNumber+1

        else:
            MatchGiveGoal =  TeamMatch.loc[indexTeam, 'away_team_goal']
            MatchTakeGoal = TeamMatch.loc[indexTeam,'home_team_goal']
            GiveGoal = GiveGoal + MatchGiveGoal
            TakeGoal = TakeGoal + MatchTakeGoal


            if MatchGiveGoal > MatchTakeGoal:
                WinNumber = WinNumber + 1

    Data.at[index,'Last_5_match_goal_diff_away '] = GiveGoal-TakeGoal
    Data.at[index,'Last_5_match_win_away  '] = WinNumber




#Away Team


for index in Data.index:
    AwayID=Data.loc[index,'Away_ID']
    HomeID=Data.loc[index,'Home_ID']

    MatchDate=Data.loc[index,'DateTime']

    BTWMatch=Match[(Match['away_team_api_id']==AwayID) &  (Match['home_team_api_id']==HomeID) |  (Match['away_team_api_id']==HomeID) &  (Match['home_team_api_id']==AwayID)]

    # BTWMatch=BTWMatch[BTWMatch['date']<MatchDate]

    Last_5_match_btw=0

    if len(BTWMatch)==0:
        pass

    else:

        if len(BTWMatch)>5:

            BTWMatch=BTWMatch.iloc[-5:]


        for  BTWMatchIndex in BTWMatch.index:

            HTG = BTWMatch.loc[BTWMatchIndex, 'home_team_goal']
            ATG = BTWMatch.loc[BTWMatchIndex, 'away_team_goal']

            if  HomeID == BTWMatch.loc[BTWMatchIndex,'home_team_api_id']:
                IsHomeHome=True

            else:
                IsHomeHome=False



            if IsHomeHome==True:

                if HTG>ATG:

                    Last_5_match_btw=Last_5_match_btw + 1

                elif HTG < ATG:
                    Last_5_match_btw = Last_5_match_btw - 1

            elif IsHomeHome==False:

                if HTG>ATG:

                    Last_5_match_btw=Last_5_match_btw -1

                elif HTG < ATG:
                    Last_5_match_btw = Last_5_match_btw + 1
    Data.at[index,'Last_5_match_btw'] = Last_5_match_btw



for index in Data.index:
    MatchInfo=Data.loc[index]
    HomeTeamPlayers= ['HomePlayer_1_Overall', 'HomePlayer_2_Overall', 'HomePlayer_3_Overall',
       'HomePlayer_4_Overall', 'HomePlayer_5_Overall', 'HomePlayer_6_Overall',
       'HomePlayer_7_Overall', 'HomePlayer_8_Overall', 'HomePlayer_9_Overall',
       'HomePlayer_10_Overall', 'HomePlayer_11_Overall']

    AwayTeamPlayers= ['AwayPlayer_1_Overall', 'AwayPlayer_2_Overall', 'AwayPlayer_3_Overall',
       'AwayPlayer_4_Overall', 'AwayPlayer_5_Overall', 'AwayPlayer_6_Overall',
       'AwayPlayer_7_Overall', 'AwayPlayer_8_Overall', 'AwayPlayer_9_Overall',
       'AwayPlayer_10_Overall', 'AwayPlayer_11_Overall']

    HomePlayerSummary=MatchInfo[HomeTeamPlayers]
    AwayPlayerSummary=MatchInfo[AwayTeamPlayers]

    Data.at[index,'Team_variance_home ']= (np.std(HomePlayerSummary))**2
    Data.at[index,'Team_Mean_home ']= (np.mean(HomePlayerSummary))
    Data.at[index,'Team_variance_away ']= (np.std(AwayPlayerSummary))**2
    Data.at[index,'Team_Mean_away ']= (np.mean(AwayPlayerSummary))




#Match Information like Card,Corner,and Shout statistics

for index in Data.index:
    HomeID=Data.loc[index,'Home_ID']
    HomeTeamName=Team[Team['team_api_id']==HomeID]['team_long_name'].iloc[0]
    MatchDay=Data.loc[index,'DateTime']
    HomeMatchData=Total[(Total['HomeTeam']==HomeTeamName) | (Total['AwayTeam']==HomeTeamName)]
    HomeMatchDataLast5=HomeMatchData[HomeMatchData['DateTime']<MatchDay].iloc[-5:]
    ShoutHome=0
    ShoutOnHome=0
    CardHome=0
    FaulHome=0
    CornerHome=0

    for MatchIndex in HomeMatchDataLast5.index:

        if HomeMatchDataLast5.loc[MatchIndex,'HomeTeam'] ==HomeTeamName :
            HomeOrAway='H'
        else:
            HomeOrAway='A'

        Shout=HomeMatchDataLast5.loc[MatchIndex,HomeOrAway+'S']
        ShoutHome=ShoutHome+Shout

        ShoutOn= HomeMatchDataLast5.loc[MatchIndex,HomeOrAway+'ST']
        ShoutOnHome=ShoutOnHome+ShoutOn

        YellowCard= HomeMatchDataLast5.loc[MatchIndex,HomeOrAway+'Y']
        RedCard= HomeMatchDataLast5.loc[MatchIndex,HomeOrAway+'R']
        CardHome=CardHome+ (2*RedCard)+ YellowCard


        Faul= HomeMatchDataLast5.loc[MatchIndex,HomeOrAway+'F']

        FaulHome=FaulHome+Faul

        Corner= HomeMatchDataLast5.loc[MatchIndex,HomeOrAway+'C']

        CornerHome=CornerHome+Corner
    Data.at[index,'Last_5_match_shout_total_home ']=ShoutHome
    Data.at[index,'Last_5_match_shout_on_home ']=ShoutOnHome
    Data.at[index,'Last_5_match_card_home ']=CardHome
    Data.at[index,'Last_5_match_faul_home ']=FaulHome
    Data.at[index,'Last_5_match_corner_home ']=CornerHome

for index in Data.index:

    AwayID = Data.loc[index, 'Away_ID']
    AwayTeamName = Team[Team['team_api_id'] == AwayID]['team_long_name'].iloc[0]
    MatchDay = Data.loc[index, 'DateTime']
    AwayMatchData = Total[(Total['AwayTeam'] == AwayTeamName) | (Total['AwayTeam'] == AwayTeamName)]
    AwayMatchDataLast5 = AwayMatchData[AwayMatchData['DateTime'] < MatchDay].iloc[-5:]
    ShoutAway = 0
    ShoutOnAway = 0
    CardAway = 0
    FaulAway = 0
    CornerAway = 0

    for MatchIndex in AwayMatchDataLast5.index:

        if AwayMatchDataLast5.loc[MatchIndex, 'HomeTeam'] == AwayTeamName:
            HomeOrAway = 'H'
        else:
            HomeOrAway = 'A'

        Shout = AwayMatchDataLast5.loc[MatchIndex, HomeOrAway + 'S']
        ShoutAway = ShoutAway + Shout

        ShoutOn = AwayMatchDataLast5.loc[MatchIndex, HomeOrAway + 'ST']
        ShoutOnAway = ShoutOnAway + ShoutOn

        YellowCard = AwayMatchDataLast5.loc[MatchIndex, HomeOrAway + 'Y']
        RedCard = AwayMatchDataLast5.loc[MatchIndex, HomeOrAway + 'R']
        CardAway = CardAway + (2 * RedCard) + YellowCard

        Faul = AwayMatchDataLast5.loc[MatchIndex, HomeOrAway + 'F']

        FaulAway = FaulAway + Faul

        Corner = AwayMatchDataLast5.loc[MatchIndex, HomeOrAway + 'C']

        CornerAway = CornerAway + Corner
    Data.at[index, 'Last_5_match_shout_total_Away '] = ShoutAway
    Data.at[index, 'Last_5_match_shout_on_Away '] = ShoutOnAway
    Data.at[index, 'Last_5_match_card_Away '] = CardAway
    Data.at[index, 'Last_5_match_faul_Away '] = FaulAway
    Data.at[index, 'Last_5_match_corner_Away '] = CornerAway

# Home Team Last Match Result


for index in Data.index:
    HomeID=Data.loc[index,'Home_ID']
    MatchDate=Data.loc[index,'DateTime']
    TeamMatch=Match[(Match['home_team_api_id']==HomeID) |  (Match['away_team_api_id']==HomeID)]
    TeamMatch=TeamMatch.sort_values(by=['date'])

    LastMatchOrder=TeamMatch[TeamMatch['date']<MatchDate].iloc[-1:]

    if len(LastMatchOrder)>0:

        if LastMatchOrder['home_team_api_id'].iloc[0] == HomeID:
            HomeTeamGaol = LastMatchOrder['home_team_goal'].iloc[0]
            RivalTeamGoal = LastMatchOrder['away_team_goal'].iloc[0]
            HomeResult=0
            if HomeTeamGaol<RivalTeamGoal:
                HomeResult = -1
            elif HomeTeamGaol>RivalTeamGoal:
                HomeResult = 1


        else:
            HomeTeamGaol = LastMatchOrder['away_team_goal'].iloc[0]
            RivalTeamGoal = LastMatchOrder['home_team_goal'].iloc[0]
            HomeResult = 0
            if HomeTeamGaol < RivalTeamGoal:
                HomeResult = -1
            elif HomeTeamGaol > RivalTeamGoal:
                HomeResult = 1

        Data.at[index,'home_team_last_result']=HomeResult
    else:
        Data.at[index,'home_team_last_result']=None



#away team last match result
for index in Data.index:
    AwayID=Data.loc[index,'Away_ID']
    MatchDate=Data.loc[index,'DateTime']
    TeamMatch=Match[(Match['home_team_api_id']==AwayID) |  (Match['away_team_api_id']==AwayID)]
    TeamMatch=TeamMatch.sort_values(by=['date'])

    LastMatchOrder=TeamMatch[TeamMatch['date']<MatchDate].iloc[-1:]

    if len(LastMatchOrder)>0:

        if LastMatchOrder['home_team_api_id'].iloc[0] == AwayID:
            AwayTeamGaol = LastMatchOrder['home_team_goal'].iloc[0]
            RivalTeamGoal = LastMatchOrder['away_team_goal'].iloc[0]
            AwayResult=0

            if AwayTeamGaol<RivalTeamGoal:
                AwayResult = -1
            elif AwayTeamGaol>RivalTeamGoal:
                AwayResult = 1

        else:
            AwayTeamGaol = LastMatchOrder['away_team_goal'].iloc[0]
            RivalTeamGoal = LastMatchOrder['home_team_goal'].iloc[0]
            AwayResult = 0
            if AwayTeamGaol < RivalTeamGoal:
                AwayResult = -1
            elif AwayTeamGaol > RivalTeamGoal:
                AwayResult = 1

        Data.at[index,'away_team_last_result']=AwayResult
    else:
        Data.at[index,'away_team_last_result']=None







for col in ColumnNames:
    del Data[col]

del Data['Away_ID']
del Data['Home_ID']




HomePlayer_1 is completed in 46.05446910858154 seconds
HomePlayer_2 is completed in 42.95395350456238 seconds
HomePlayer_3 is completed in 40.273316860198975 seconds
HomePlayer_4 is completed in 40.495171308517456 seconds
HomePlayer_5 is completed in 40.82726192474365 seconds
HomePlayer_6 is completed in 44.6092369556427 seconds
HomePlayer_7 is completed in 53.89186716079712 seconds
HomePlayer_8 is completed in 48.73768067359924 seconds
HomePlayer_9 is completed in 41.322582960128784 seconds
HomePlayer_10 is completed in 40.67425537109375 seconds
HomePlayer_11 is completed in 40.5923969745636 seconds
AwayPlayer_1 is completed in 44.167900800704956 seconds
AwayPlayer_2 is completed in 45.22504734992981 seconds
AwayPlayer_3 is completed in 41.50300168991089 seconds
AwayPlayer_4 is completed in 41.203829526901245 seconds
AwayPlayer_5 is completed in 41.339439392089844 seconds
AwayPlayer_6 is completed in 41.54587745666504 seconds
AwayPlayer_7 is completed in 42.57413721084595 seconds
Away

In [2]:
Data=Data.dropna()

In [3]:
Data.to_excel('FinalDataSet.xlsx')